# JPK AFM data analysis for liquid samples

## Import libraries
Run this ONLY ONCE. Always restart kernel before running.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime
from igor import binarywave
from PyQt5.QtWidgets import QApplication, QFileDialog, QListView, QTreeView, QAbstractItemView
#from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import wetting#, surfevol

app = QApplication(sys.argv)
plt.close('all')

#function to select multiple folders from dialog
def get_directories(caption=""):
    file_dialog = QFileDialog(caption=caption)
    file_dialog.setFileMode(QFileDialog.DirectoryOnly)
    file_dialog.setOption(QFileDialog.DontUseNativeDialog, True)
    file_view = file_dialog.findChild(QListView, 'listView')

    if file_view:
        file_view.setSelectionMode(QAbstractItemView.MultiSelection)
    f_tree_view = file_dialog.findChild(QTreeView)
    if f_tree_view:
        f_tree_view.setSelectionMode(QAbstractItemView.MultiSelection)

    if file_dialog.exec():
        paths = file_dialog.selectedFiles()
    else:
        paths = []

    return paths

#recognize if *.ibw file contains image data or force data
def categorize_ibw(filepath):
    ibw = binarywave.load(filepath)
    wdata = ibw["wave"]["wData"]
    meta_keys = []
    for line in str(ibw["wave"]["note"]).split("\\r"):
        if line.count(":"):
            key, val = line.split(":", 1)
            meta_keys.append(key)

    #CHECK THIS! MIGHT NOT WORK FOR SOME DATA
    if 'ForceDist' in meta_keys:
        file_cat = 'force'
    else:
        file_cat = 'image'
        
    return file_cat        

## Run Surface Tension code
Only works with classic Jupyter Notebook, not JupyterLab

In [ ]:
%%javascript
Jupyter.notebook.execute_cells([6,8,10,12,14])

## JPK data raw files

In [ ]:
jpk_file_paths, _ = QFileDialog.getOpenFileNames(caption='Select JPK data files') #JPK data (*.jpk,*.jpk-qi-data,*.jpk-force)
output_dir = ''#QFileDialog.getExistingDirectory(caption='Select output data folder') #output folder
        
#separate image data and force data files
img_file_paths = []
fd_file_paths = []
for filepath in jpk_file_paths:
    file_ext = filepath.split('.')[-1]
    if file_ext in ['jpk', 'jpk-qi-data', 'jpk-force-map']:
        img_file_paths.append(filepath)
    elif file_ext in ['jpk-force']:
        fd_file_paths.append(filepath)
    elif file_ext in ['ibw']: #change this
        file_cat = categorize_ibw(filepath)
        if file_cat == 'image':
            img_file_paths.append(filepath)
        elif file_cat == 'force':
            fd_file_paths.append(filepath)
        
#make output directory
if output_dir == '':
        output_dir = os.path.dirname(jpk_file_paths[0]) + '/analysis' #default "analysis" folder
output_paths = []
for img_file_path in img_file_paths:
    file_name = os.path.basename(img_file_path)
    timestamp = datetime.today().strftime('%y%m%d-%H%M%S')
    output_path = f'{output_dir}/{file_name}_results_{timestamp}'
    os.makedirs(output_path, exist_ok=True)
    output_paths.append(output_path)

print('Image data:\n', img_file_paths, '\nForce data:\n', fd_file_paths, '\nOutput folder:\n', output_paths)

## AFM image

In [ ]:
plt.close('all')
for img_file_path, output_path in zip(img_file_paths, output_paths):
    print('Image file:', img_file_path)
    print('Output folder:', output_path)
    afm_data, anal_data_h, fig_list =  wetting.get_afm_image(img_file_path, output_path, 
                                                             level_order=2, jump_tol=0.9,
                                                             denoise_size=5)
    for fig in fig_list:
        display(fig)

plt.close('all')

## Get liquid drop properties

In [ ]:
plt.close('all')
#drop analysis of AFM data
drop_df, img_anal, fig_list = wetting.get_drop_prop(afm_data, anal_data_h, output_paths[0])
drop_df['AFM file'] = img_file_paths[0]
for fig in fig_list:
    display(fig)
display(drop_df)
plt.close('all')

## Analyze force distance curves

In [ ]:
plt.close('all')
fd_drop_df, fdfit_dict, fddata_dict, fig_list = wetting.analyze_drop_fd(fd_file_paths, afm_data, img_anal,
                                                                        force_cycle = 'approach', fit_order = 2,
                                                                        output_path = output_paths[0])

for fig in fig_list:
    #fig = Image(fig_path, width=400, height=400)
    display(fig)
display(fd_drop_df)

## Droplet surface tension calculation

In [ ]:
contact_angle = 40 #Set fixed value to calculate its corresponsing surface tension (None take average of polyfit contact angle values)
cone_a = 29 # cone angle from SEM (near tip)
pyr_a = 38 #pyramid angle from SEM

pyr_a_eff = 2*np.arctan(np.tan((pyr_a/2)*np.pi/180/2)/np.sqrt(2))*180/np.pi #effective half angle since SEM images are oriented 45
tip_angle_dict = {'Cone':int(cone_a/2), 
                  'Pyramid':int(pyr_a_eff)}

plt.close('all')
sns.set_style("ticks")
sns.set_context("talk")

#combine droplet image and force result data
afm_df = drop_df.set_index('Label').join(fd_drop_df.set_index('Label'), how='right')
    
# calculate surface tension by complete FD fitting using 2nd order poly approximation
output_df_cone2, fig_fit_cone = wetting.get_surface_tension(afm_df, 'Cone', tip_angle_dict['Cone'], 
                                               contact_angle, fdfit_dict, fddata_dict,
                                               file_path=output_paths[0], save=False)
output_df_pyr2, fig_fit_pyr = wetting.get_surface_tension(afm_df, 'Pyramid', tip_angle_dict['Pyramid'], 
                                               contact_angle, fdfit_dict, fddata_dict,
                                               file_path=output_paths[0], save=False)
output_df_polyfit = output_df_cone2.append(output_df_pyr2)
afm_filename = output_df_polyfit['AFM file'].iloc[0].split('/')[-1][:-4]
output_df_polyfit.to_excel(f'{output_paths[0]}/output_final-{afm_filename}.xlsx')

#display fitting plots
display(fig_fit_cone)
fig_fit_cone.savefig(f'{output_paths[0]}/cone_polyfit_result.png', bbox_inches = 'tight',
            transparent = False)
display(fig_fit_pyr)
fig_fit_pyr.savefig(f'{output_paths[0]}/pyramid_polyfit_result.png', bbox_inches = 'tight',
            transparent = False)
plt.close('all')

#plot surface tension results
output_df_polyfit_reshaped = pd.melt(output_df_polyfit[['Tip shape','Contact Radius','Surface Tension (polyfit, mN/m)','Surface Tension (fixed, mN/m)']], 
                                     id_vars=['Tip shape','Contact Radius'],
                                    value_vars=['Surface Tension (polyfit, mN/m)','Surface Tension (fixed, mN/m)'],
                                    var_name='Method', value_name='Surface Tension (mN/m)')
output_df_polyfit_reshaped.replace({'Method': {'Surface Tension (polyfit, mN/m)': 'polyfit', 
                                               'Surface Tension (fixed, mN/m)': 'fixed'}}, inplace=True)
g1 = sns.relplot(data=output_df_polyfit_reshaped, x="Contact Radius", 
                      y="Surface Tension (mN/m)",
                     style="Tip shape",hue="Method",
                     kind='scatter', aspect=1.3)
#show and save plot
fig1 = g1.figure
display(fig1)
fig1.savefig(f'{output_paths[0]}/surface_tension.png', bbox_inches = 'tight',
             transparent = False)


plot_vars = ['Drop contact angle','Tip contact angle (polyfit)',
             'Adhesion (FD)', 'Max Height']

fig2, ax2 = plt.subplots(2, 2, sharex=True, figsize=(12, 8))
sns.scatterplot(data=output_df_polyfit, x="Contact Radius",
                y=plot_vars[0], ax=ax2[0,0])
sns.scatterplot(data=output_df_polyfit, x="Contact Radius",
                y=plot_vars[1], style="Tip shape", hue="Tip shape", ax=ax2[0,1])
sns.scatterplot(data=output_df_polyfit, x="Contact Radius",
                y=plot_vars[2], ax=ax2[1,0])
sns.scatterplot(data=output_df_polyfit, x="Contact Radius",
                y=plot_vars[3], ax=ax2[1,1])

#number formatting
ax2[1][0].ticklabel_format(axis='y', style='sci', scilimits=(-9,-9))
ax2[1][1].ticklabel_format(axis='y', style='sci', scilimits=(-6,-6))
ax2[1][1].ticklabel_format(axis='x', style='sci', scilimits=(-6,-6))
fig2.tight_layout()
display(fig2)
fig2.savefig(f'{output_paths[0]}/other_results.png', bbox_inches = 'tight',
             transparent = False)

plt.close('all')

## Combine results from different folders

In [ ]:
plt.cla()
plt.clf()
plt.close('all')
folder_paths = get_directories("Select analysis result folder (containing output Excel file)")
summary_df = wetting.combine_result_spreadsheets(folder_paths)

summary_df['Liquid'] = 'Glycerol' #SET THIS!
summary_df['DQ'] = 'Y' #CHANGE to N manually if data not good
summary_df = summary_df[summary_df['FD file'] != '']

summary_df.rename(columns={"Max Height": "Drop Height (m)"}, inplace=True)
#summary_df = summary_df[summary_df[surf_ten_label] != 0]
#summary_df.replace(0, np.nan, inplace=True)
timestamp = datetime.today().strftime('%y%m%d-%H%M%S')
output_dir2 = os.path.dirname(folder_paths[-1])
#dir_name = os.path.basename(folder_paths[-1]) # use this to use same name as source folder name
dir_name = os.path.basename(os.path.dirname(output_dir2))
summary_df.to_excel(f'{output_dir2}/summary_data_{dir_name}_{timestamp}.xlsx', index=None)


#plot surface tension results
x_var = "Drop Height (m)"# "Contact Radius",'Liquid','R square (polyfit)',
val_vars = ['Surface Tension (polyfit, mN/m)','Surface Tension (fixed, mN/m)']
id_vars = [p for p in list(summary_df.columns) if p not in val_vars]
cols = summary_df.columns
summary_df_reshaped = pd.melt(summary_df, 
                                     id_vars=id_vars,
                                    value_vars=val_vars,
                                    var_name='Method', value_name='Surface Tension (mN/m)')
summary_df_reshaped.replace({'Method': {'Surface Tension (polyfit, mN/m)': 'polyfit', 
                                        'Surface Tension (fixed, mN/m)': 'fixed'}}, inplace=True)
#summary_df_reshaped = summary_df_reshaped[summary_df_reshaped['Method'] == 'fixed'] #CHECK
#summary_df_reshaped = summary_df_reshaped[summary_df_reshaped['DQ'] == 'Y'] #CHECK

# #summary_df_reshaped = summary_df_reshaped[summary_df_reshaped['R square (polyfit)'] > 0.9]
# #summary_df_reshaped = summary_df_reshaped[summary_df_reshaped['Tip shape'] == 'Cone']
#group data by tip shape to get average surface tension
# summary_df_reshaped = summary_df_reshaped[['Drop Height (m)','Liquid','Tip shape',
#                                            'Surface Tension (mN/m)']]
# summary_df_reshaped = summary_df_reshaped.groupby(['Drop Height (m)','Liquid']).mean()

#generate pivot table to summarize surface tension for fixed contact angle
summary_final = pd.pivot_table(summary_df_reshaped[summary_df_reshaped['Method'] == 'fixed'], 
                               values=['Surface Tension (mN/m)'], 
                               index=['Liquid','Tip shape'],
                               aggfunc={'Surface Tension (mN/m)': [np.mean, np.std]})
summary_final.to_excel(f'{output_dir2}/summary_final_{dir_name}_{timestamp}.xlsx')
display(summary_final)

g3 = sns.relplot(data=summary_df_reshaped, x=x_var,
                y="Surface Tension (mN/m)", hue="Method", #Method, Liquid CHECK
                style="Tip shape",
                style_order=['Cone','Pyramid'],
                kind='scatter', aspect=1.3)

#ax.ticklabel_format(axis='x', style='sci', scilimits=(-6,-6))
#show and save plot
fig3 = g3.figure
fig3.savefig(f'{output_dir2}/summary_plot_{dir_name}_{timestamp}.png', bbox_inches = 'tight',
            transparent = False)
display(fig3)

plot_vars = ['Drop contact angle','Tip contact angle (polyfit)',
             'Adhesion (FD)', 'Drop Height (m)']

fig4, ax4 = plt.subplots(2, 2, sharex=True, figsize=(12, 8))
sns.scatterplot(data=summary_df, x=x_var,
                y=plot_vars[0], ax=ax4[0,0])
sns.scatterplot(data=summary_df, x=x_var,
                y=plot_vars[1], style="Tip shape", 
                hue="Liquid", 
                ax=ax4[0,1])
sns.scatterplot(data=summary_df, x=x_var,
                y=plot_vars[2], ax=ax4[1,0])
sns.scatterplot(data=summary_df, x=x_var,
                y=plot_vars[3], ax=ax4[1,1])

#number formatting
ax4[1][0].ticklabel_format(axis='y', style='sci', scilimits=(-9,-9))
ax4[1][1].ticklabel_format(axis='y', style='sci', scilimits=(-6,-6))
ax4[1][1].ticklabel_format(axis='x', style='sci', scilimits=(-6,-6))
fig4.tight_layout()
display(fig4)
fig4.savefig(f'{output_dir2}/summary_other_results_{dir_name}_{timestamp}.png', bbox_inches = 'tight',
             transparent = False)

display(summary_df)
plt.close('all')

## Combine multiple FD curves

In [ ]:
plt.close('all')
# combine multiple fd curves
output_path = ''
fd_file_paths, _ = QFileDialog.getOpenFileNames()
wetting.combine_fd(fd_file_paths, zero_shift=True, 
                   output_dir=output_path,save=False)
#wetting.get_adhesion_from_fd(fd_file_paths)

## Optional codes

In [ ]:
# calculate contact angle from fd curve
label = 5 #INPUT
label_df = drop_df[drop_df['Label']==label]
s = label_df['s'].iloc[0]
R = round(label_df['R/s'].iloc[0])
contact_angle = wetting.get_contact_angle(fd_file_paths[0], simu_df,
                                         R, s, fit_index=5000)

